<a href="https://colab.research.google.com/github/RakshithaMraksh/python_projects/blob/main/SQL_Command_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch transformers bitsandbytes accelerate sqlparse

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 6.0 MB/s eta 0:00:00


In [9]:
import torch
from transformers import AutoTokenizer , AutoModelForCausalLM

In [4]:
torch.cuda.is_available()

True

In [11]:
available_memory = torch.cuda.get_device_properties(0).total_memory

In [6]:
print(available_memory)

15835660288


In [12]:
model_name = "defog/sqlcoder-7b-2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
if available_memory > 1669:
      #if you have atleast 15GB of GPU memory, run load the model in float16
      model = AutoModelForCausalLM.from_pretrained(
      model_name,
      trust_remote_code=True,
      torch_dtype=torch.float16, device_map="auto",
      use_cache=True,
)
else:
        #else, load in 8 bits this is a bit slower
      model_name,
      model = AutoModelForCausalLM.from_pretrained(
          trust_remote_code=True,
          #torch_dtype=torch.float16,
          load_in_8bit=True, device_map="auto",
          use_cache=True,
    )

config.json:   0%|          | 0.00/691 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

**Set the question & Prompt and Tokenize**

In [13]:
prompt = """ ### Task
Generate a SQL query to answer [QUESTION]{question}[/QUESTION]

### Instruction
- If you cannot answer the question with the available database schema, return 'I dont know'
- Remember that revenue is price multiplied by quantity
- Remember that cost is supply_price multiplied by quantity

### Database Schema
This query will run on a database whose schema is represented in this string:
CREATE TABLE products (
  product_id INTEGER PRIMARY KEY, --Unique ID for each product
  name VARCHAR(50), --Name of the product
  price DECIMAL(10,2), -- Price of each unit of the product
  quantity INTEGER --Current quantity in stock
);

CREATE TABLE customers (
  customer_id INTEGER PRIMARY KEY, --Unique ID for each customer
  name VARCHAR(50), --Name of the customer
  address VARCHAR(100) --Mailing address of the customer
);

CREATE TABLE salespeople (
  salesperson_id INTEGER PRIMARY KEY, --Unique ID for each salesperson
  name VARCHAR(50), --Name of the salesperson
  region VARCHAR(50) --Geographic sales region
);

CREATE TABLE sales (
  sale_id INTEGER PRIMARY KEY, --Unique ID for each sale
  product_id INTEGER, --ID of product sold
  customer_id INTEGER, --ID of customer who made purchase
  salesperson_id INTEGER, --ID of salesperson who made the sale
  sale_date DATE, Date the sale occurred
  quantity INTEGER Quantity of product sold
);

CREATE TABLE product_suppliers (
  supplier_id INTEGER PRIMARY KEY, --Unique ID for each supplier
  product_id INTEGER, --Product ID supplied
  supply_price DECIMAL(10,2) --Unit price charged by supplier
);

--sales.product_id can be joined with products.product_id
--sales.customer_id can be joined with customers.customer_id
--sales.salesperson_id can be joined with salespeople.salesperson_id

product_suppliers.product_id can be joined with products.product_id

### Answer
Given the database schema, here is the SQL query that answers [QUESTION] (question) [/QUESTION)
[SQL]
"""



In [16]:
import sqlparse
prompt = "Generate an SQL query based on this question: {question}"

def generate_query(question):
  updated_prompt = prompt.format(question=question)
  inputs = tokenizer (updated_prompt, return_tensors="pt").to("cuda")
  generated_ids= model.generate(
      **inputs,
      num_return_sequences=1,
      eos_token_id=tokenizer.eos_token_id,
      pad_token_id=tokenizer.eos_token_id,
      max_new_tokens=400,
      do_sample=False,
      num_beams=1,
  )
  outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

  torch.cuda.empty_cache()
  torch.cuda.synchronize()
  # empty cache so that you do generate more results w/o memory crashing
  #particularly important on Colab memory management is much more straightforward
  #when running on an inference service
  return sqlparse.format(outputs[0].split("[SQL]") [-1], reindent=True)

In [17]:
question = "What was the highest quantity sold last month?"
generated_sql = generate_query(question)
print(generated_sql)

Generate an SQL query based on this question: What was the highest quantity sold last month? I have a table with sales data,
and I want to know what was the highest quantity sold last month. I have tried this query: \begin{code}
SELECT MAX(s.quantity) AS max_quantity
FROM sales s
WHERE s.date_sold >= (CURRENT_DATE - INTERVAL '1 month');

\end{code} But I get an error: \begin{code} ERROR: syntax error at
or near "1" LINE 1:
SELECT MAX(s.quantity) AS max_quantity
FROM sales s
WHERE s.date_sold >= (CURRENT_DATE - INTERVAL '1 month');

\end{code} What am I doing wrong? Answer: You need to use single quotes instead of double quotes: \begin{code}
SELECT MAX(s.quantity) AS max_quantity
FROM sales s
WHERE s.date_sold >= (CURRENT_DATE - INTERVAL '1 month');


In [18]:
question = "Which salesperson sold large amount of products last month?"

generated_sql = generate_query(question)

print(generated_sql)

Generate an SQL query based on this question: Which salesperson sold large amount of products last month? I have a sales table with the following columns: salesperson, date, product,
                                                                                                                                                                              quantity. I want to generate an SQL query that returns the salesperson who sold the most products last month. I have tried the following query:
SELECT salesperson,
       SUM(quantity) AS total_quantity
FROM sales
WHERE date >= (CURRENT_DATE - INTERVAL '1 month')
GROUP BY salesperson
ORDER BY total_quantity DESC
LIMIT 1;


In [19]:
question = "What was our revenue by product in the new york region last month?"
generated_sql = generate_query(question)